In [3]:
import pandas as pd
import numpy as np
import gc
import pickle

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

import lightgbm as lgb

from bayes_opt import BayesianOptimization

## Set random number seed, read in final dataset (1293 variables initially but reduced to 961) 

In [82]:
seed=123
X=pd.read_pickle('loan data_961.pkl')
Y=X.join(pd.read_csv("application_train.csv")[['SK_ID_CURR','TARGET']].set_index('SK_ID_CURR').TARGET,how='left').TARGET

## Open optimized parameters file

In [4]:
pkl_file = open('lgb_params_961.pkl', 'rb')
lgb_params = pickle.load(pkl_file)
pkl_file.close()

In [6]:
lgb_params['params']

{'application': 'binary',
 'bagging_fraction': 0.12328510390504416,
 'boosting': 'gbdt',
 'early_stopping_round': 100,
 'feature_fraction': 0.186302213744303,
 'lambda_l1': 4.8511826178831985,
 'lambda_l2': 0.20632914438356242,
 'learning_rate': 0.01,
 'max_bin': 224,
 'max_depth': 19,
 'metric': 'binary_logloss',
 'min_child_weight': 7.681400599469479,
 'min_data_in_leaf': 175,
 'min_split_gain': 0.02516279405301516,
 'num_iteration': 1814,
 'num_leaves': 23}

## Create train & test datasets

In [86]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=seed)

## Run 120 iterations of bayesian optimization to find hyperparameters

In [4]:
def bayes_parameter_opt_lgb(X_train, Y_train, init_round, opt_round, n_folds, random_seed, num_iterations, learning_rate):
    
    train_data = lgb.Dataset(data=X_train,label=Y_train)
    
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight,min_data_in_leaf):
        params = {'application':'binary','num_iterations':num_iterations, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (2, 100),
                                            'feature_fraction': (0.05, 0.95),
                                            'bagging_fraction': (0.05, .95),
                                            'max_depth': (4, 20),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 5),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 100),
                                            'min_data_in_leaf': (50,250)}, random_state=random_seed)
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    print(lgbBO.max)
    return lgbBO

lgbBO = bayes_parameter_opt_lgb(X_train, Y_train, init_round=40, opt_round=80, n_folds=3, random_seed=seed, num_iterations=4000, learning_rate=0.01)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------


C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:431: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.771903 + 0.00193865
[400]	cv_agg's auc: 0.779365 + 0.0021716
[600]	cv_agg's auc: 0.783126 + 0.0019843
[800]	cv_agg's auc: 0.784925 + 0.00200017
[1000]	cv_agg's auc: 0.785772 + 0.0020359
[1200]	cv_agg's auc: 0.78609 + 0.00205108
[1400]	cv_agg's auc: 0.786402 + 0.00208456
|  1        |  0.7864   |  0.6768   |  0.3075   |  1.134    |  2.757    |  15.51    |  45.2     |  246.2    |  0.0688   |  49.13    |
[200]	cv_agg's auc: 0.765285 + 0.00199422
[400]	cv_agg's auc: 0.774339 + 0.00186146
[600]	cv_agg's auc: 0.779188 + 0.00174379
[800]	cv_agg's auc: 0.782022 + 0.00193756
[1000]	cv_agg's auc: 0.783641 + 0.00196865
[1200]	cv_agg's auc: 0.784701 + 0.00204626
[1400]	cv_agg's auc: 0.785236 + 0.00217058
[1600]	cv_agg's auc: 0.785587 + 0.00218605
[1800]	cv_agg's auc: 0.7859 + 0.00217488
[2000]	cv_agg's auc: 0.786123 + 0.00226463
[2200]	cv_agg's auc: 0.786241 + 0.00235439
[2400]	cv_agg's auc: 0.786327 + 0.00238896
[2600]	cv_agg's auc: 0.786348 + 0.00243174
|  2        |  0.786

[200]	cv_agg's auc: 0.772679 + 0.00219868
[400]	cv_agg's auc: 0.780212 + 0.00206961
[600]	cv_agg's auc: 0.783821 + 0.00206631
[800]	cv_agg's auc: 0.785318 + 0.00227328
[1000]	cv_agg's auc: 0.785979 + 0.00221561
[1200]	cv_agg's auc: 0.786171 + 0.00229145
|  16       |  0.7862   |  0.1214   |  0.4355   |  1.023    |  2.253    |  12.76    |  13.87    |  109.4    |  0.09283  |  57.76    |
[200]	cv_agg's auc: 0.76419 + 0.00217876
[400]	cv_agg's auc: 0.7767 + 0.00261038
[600]	cv_agg's auc: 0.781549 + 0.00285586
[800]	cv_agg's auc: 0.783811 + 0.00287673
[1000]	cv_agg's auc: 0.784884 + 0.00288735
[1200]	cv_agg's auc: 0.785316 + 0.00280094
[1400]	cv_agg's auc: 0.785329 + 0.00275034
|  17       |  0.7854   |  0.4617   |  0.7282   |  3.709    |  0.2429   |  15.34    |  84.73    |  83.19    |  0.07832  |  30.08    |
[200]	cv_agg's auc: 0.768545 + 0.00212225
[400]	cv_agg's auc: 0.778836 + 0.0024941
[600]	cv_agg's auc: 0.783166 + 0.00261747
[800]	cv_agg's auc: 0.784792 + 0.00272812
[1000]	cv_agg's a

[1600]	cv_agg's auc: 0.784144 + 0.00251083
|  30       |  0.7842   |  0.1107   |  0.9437   |  1.182    |  1.871    |  7.424    |  15.02    |  96.5     |  0.03076  |  64.18    |
[200]	cv_agg's auc: 0.773515 + 0.00164445
[400]	cv_agg's auc: 0.780173 + 0.00174988
[600]	cv_agg's auc: 0.783563 + 0.00191328
[800]	cv_agg's auc: 0.785157 + 0.00191827
[1000]	cv_agg's auc: 0.785729 + 0.00192471
|  31       |  0.7858   |  0.3031   |  0.376    |  0.02971  |  1.829    |  12.54    |  20.39    |  169.5    |  0.03002  |  63.94    |
[200]	cv_agg's auc: 0.769046 + 0.00201722
[400]	cv_agg's auc: 0.780118 + 0.00208196
[600]	cv_agg's auc: 0.783576 + 0.00226678
[800]	cv_agg's auc: 0.784703 + 0.00234449
[1000]	cv_agg's auc: 0.785029 + 0.00243911
|  32       |  0.7851   |  0.07358  |  0.8488   |  0.08059  |  0.6348   |  16.43    |  9.36     |  192.2    |  0.09713  |  87.42    |
[200]	cv_agg's auc: 0.764455 + 0.00230224
[400]	cv_agg's auc: 0.777013 + 0.00265471
[600]	cv_agg's auc: 0.781608 + 0.00294266
[800]	c

...

[200]	cv_agg's auc: 0.755663 + 0.000995782
[400]	cv_agg's auc: 0.769436 + 0.00145328
[600]	cv_agg's auc: 0.775306 + 0.00168063
[800]	cv_agg's auc: 0.778677 + 0.0017272
[1000]	cv_agg's auc: 0.780967 + 0.00170198
[1200]	cv_agg's auc: 0.782587 + 0.0017966
[1400]	cv_agg's auc: 0.783502 + 0.0017757
[1600]	cv_agg's auc: 0.784231 + 0.00185317
[1800]	cv_agg's auc: 0.784708 + 0.00187395
[2000]	cv_agg's auc: 0.78513 + 0.00188988
[2200]	cv_agg's auc: 0.785482 + 0.00195249
[2400]	cv_agg's auc: 0.785752 + 0.00201311
[2600]	cv_agg's auc: 0.785937 + 0.00200847
[2800]	cv_agg's auc: 0.786063 + 0.00210106
[3000]	cv_agg's auc: 0.786176 + 0.00214928
[3200]	cv_agg's auc: 0.786243 + 0.00219538
[3400]	cv_agg's auc: 0.786319 + 0.00222087
|  118      |  0.7863   |  0.2143   |  0.1064   |  4.839    |  0.2541   |  4.243    |  58.64    |  175.8    |  0.09712  |  55.45    |


C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:431: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.736208 + 0.00358679
[400]	cv_agg's auc: 0.752458 + 0.00282526
[600]	cv_agg's auc: 0.760751 + 0.00231875
[800]	cv_agg's auc: 0.766022 + 0.00205877
[1000]	cv_agg's auc: 0.769324 + 0.00204715
[1200]	cv_agg's auc: 0.771803 + 0.00197843
[1400]	cv_agg's auc: 0.77384 + 0.0018951
[1600]	cv_agg's auc: 0.775521 + 0.00195206
[1800]	cv_agg's auc: 0.776829 + 0.00198189
[2000]	cv_agg's auc: 0.777901 + 0.00204435
[2200]	cv_agg's auc: 0.778795 + 0.00203777
[2400]	cv_agg's auc: 0.779434 + 0.00196828
[2600]	cv_agg's auc: 0.780068 + 0.00195561
[2800]	cv_agg's auc: 0.780617 + 0.00198573
[3000]	cv_agg's auc: 0.781109 + 0.00199756
[3200]	cv_agg's auc: 0.78149 + 0.00197317
[3400]	cv_agg's auc: 0.781868 + 0.00197248
[3600]	cv_agg's auc: 0.782175 + 0.00195802
[3800]	cv_agg's auc: 0.782458 + 0.00196922
[4000]	cv_agg's auc: 0.782709 + 0.00197269
|  119      |  0.7827   |  0.6808   |  0.07647  |  4.823    |  0.05312  |  4.486    |  15.6     |  52.34    |  0.06848  |  2.525    |


C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:431: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.759664 + 0.00195386
[400]	cv_agg's auc: 0.771671 + 0.00191149
[600]	cv_agg's auc: 0.777604 + 0.00214562
[800]	cv_agg's auc: 0.780744 + 0.00202824
[1000]	cv_agg's auc: 0.782677 + 0.00208023
[1200]	cv_agg's auc: 0.784001 + 0.00205952
[1400]	cv_agg's auc: 0.784741 + 0.00196255
[1600]	cv_agg's auc: 0.785161 + 0.00199313
[1800]	cv_agg's auc: 0.785495 + 0.0019916
[2000]	cv_agg's auc: 0.785768 + 0.00200679
[2200]	cv_agg's auc: 0.785918 + 0.00206686
[2400]	cv_agg's auc: 0.78609 + 0.00207692
[2600]	cv_agg's auc: 0.786158 + 0.00216937
|  120      |  0.7862   |  0.869    |  0.1425   |  0.1789   |  0.5621   |  4.357    |  36.26    |  183.9    |  0.08792  |  59.02    |
{'target': 0.7869651597168584, 'params': {'bagging_fraction': 0.8047856904584731, 'feature_fraction': 0.18290784176366603, 'lambda_l1': 4.871762762111217, 'lambda_l2': 0.28153772033191415, 'max_depth': 5.967090576845315, 'min_child_weight': 8.205668273741352, 'min_data_in_leaf': 50.58714920463898, 'min_split_gai

## Perform 5 fold cross validation on train data

In [87]:
skf = StratifiedKFold(5, random_state=seed)

lgb_train = lgb.Dataset(data=X_train,label=Y_train)

lgb_test = lgb.Dataset(data=X_test,label=Y_test)

cv_results = lgb.cv(
    lgb_params['params'], 
    lgb_train, 
    metrics=['auc'], 
    folds=skf.split(X_train,Y_train), 
    verbose_eval=25,
)

print(cv_results['auc-mean'][-1])

C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:426: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:431: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[25]	cv_agg's auc: 0.749542 + 0.00523702
[50]	cv_agg's auc: 0.75306 + 0.00568521
[75]	cv_agg's auc: 0.757164 + 0.0058994
[100]	cv_agg's auc: 0.760653 + 0.00580338
[125]	cv_agg's auc: 0.763041 + 0.00567625
[150]	cv_agg's auc: 0.765228 + 0.00567493
[175]	cv_agg's auc: 0.766523 + 0.00596545
[200]	cv_agg's auc: 0.768671 + 0.00618957
[225]	cv_agg's auc: 0.769687 + 0.00632531
[250]	cv_agg's auc: 0.771034 + 0.00634873
[275]	cv_agg's auc: 0.772749 + 0.00642047
[300]	cv_agg's auc: 0.774317 + 0.00636353
[325]	cv_agg's auc: 0.775822 + 0.00632754
[350]	cv_agg's auc: 0.776845 + 0.0063532
[375]	cv_agg's auc: 0.778041 + 0.00637178
[400]	cv_agg's auc: 0.778761 + 0.00637485
[425]	cv_agg's auc: 0.779832 + 0.00646591
[450]	cv_agg's auc: 0.780713 + 0.00648847
[475]	cv_agg's auc: 0.781418 + 0.00652028
[500]	cv_agg's auc: 0.782102 + 0.0065119
[525]	cv_agg's auc: 0.78293 + 0.00656811
[550]	cv_agg's auc: 0.783612 + 0.00657137
[575]	cv_agg's auc: 0.784221 + 0.0065866
[600]	cv_agg's auc: 0.784756 + 0.00652404
[

## Train model to minimize binary log_loss metric

In [88]:
lgb_params['params']['num_iteration'] = int(len(cv_results['auc-mean']))*5/4

model = lgb.train(lgb_params['params'],lgb_train,num_boost_round=10,valid_sets=lgb_test)

C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\sebastian\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.692124
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.691453
[3]	valid_0's binary_logloss: 0.690785
[4]	valid_0's binary_logloss: 0.689737
[5]	valid_0's binary_logloss: 0.688946
[6]	valid_0's binary_logloss: 0.687996
[7]	valid_0's binary_logloss: 0.686937
[8]	valid_0's binary_logloss: 0.68626
[9]	valid_0's binary_logloss: 0.685631
[10]	valid_0's binary_logloss: 0.685039
[11]	valid_0's binary_logloss: 0.684015
[12]	valid_0's binary_logloss: 0.683377
[13]	valid_0's binary_logloss: 0.682324
[14]	valid_0's binary_logloss: 0.681746
[15]	valid_0's binary_logloss: 0.681134
[16]	valid_0's binary_logloss: 0.680236
[17]	valid_0's binary_logloss: 0.679501
[18]	valid_0's binary_logloss: 0.678783
[19]	valid_0's binary_logloss: 0.677861
[20]	valid_0's binary_logloss: 0.676966
[21]	valid_0's binary_logloss: 0.676436
[22]	valid_0's binary_logloss: 0.67583
[23]	valid_0's binary_logloss: 0.674962
[24]	valid_0's binary_loglos

[205]	valid_0's binary_logloss: 0.602066
[206]	valid_0's binary_logloss: 0.60187
[207]	valid_0's binary_logloss: 0.601717
[208]	valid_0's binary_logloss: 0.601539
[209]	valid_0's binary_logloss: 0.601348
[210]	valid_0's binary_logloss: 0.601156
[211]	valid_0's binary_logloss: 0.600978
[212]	valid_0's binary_logloss: 0.60075
[213]	valid_0's binary_logloss: 0.60055
[214]	valid_0's binary_logloss: 0.600393
[215]	valid_0's binary_logloss: 0.600053
[216]	valid_0's binary_logloss: 0.599867
[217]	valid_0's binary_logloss: 0.599745
[218]	valid_0's binary_logloss: 0.599576
[219]	valid_0's binary_logloss: 0.599376
[220]	valid_0's binary_logloss: 0.599206
[221]	valid_0's binary_logloss: 0.599067
[222]	valid_0's binary_logloss: 0.59895
[223]	valid_0's binary_logloss: 0.598612
[224]	valid_0's binary_logloss: 0.598454
[225]	valid_0's binary_logloss: 0.598283
[226]	valid_0's binary_logloss: 0.59813
[227]	valid_0's binary_logloss: 0.597952
[228]	valid_0's binary_logloss: 0.597766
[229]	valid_0's binar

[411]	valid_0's binary_logloss: 0.575419
[412]	valid_0's binary_logloss: 0.575357
[413]	valid_0's binary_logloss: 0.575292
[414]	valid_0's binary_logloss: 0.575232
[415]	valid_0's binary_logloss: 0.575075
[416]	valid_0's binary_logloss: 0.574927
[417]	valid_0's binary_logloss: 0.574779
[418]	valid_0's binary_logloss: 0.574727
[419]	valid_0's binary_logloss: 0.574655
[420]	valid_0's binary_logloss: 0.574599
[421]	valid_0's binary_logloss: 0.574534
[422]	valid_0's binary_logloss: 0.574462
[423]	valid_0's binary_logloss: 0.574438
[424]	valid_0's binary_logloss: 0.57439
[425]	valid_0's binary_logloss: 0.574317
[426]	valid_0's binary_logloss: 0.57425
[427]	valid_0's binary_logloss: 0.574152
[428]	valid_0's binary_logloss: 0.574095
[429]	valid_0's binary_logloss: 0.574003
[430]	valid_0's binary_logloss: 0.573872
[431]	valid_0's binary_logloss: 0.573737
[432]	valid_0's binary_logloss: 0.573676
[433]	valid_0's binary_logloss: 0.57362
[434]	valid_0's binary_logloss: 0.573584
[435]	valid_0's bin

[617]	valid_0's binary_logloss: 0.563651
[618]	valid_0's binary_logloss: 0.563629
[619]	valid_0's binary_logloss: 0.563593
[620]	valid_0's binary_logloss: 0.563563
[621]	valid_0's binary_logloss: 0.563517
[622]	valid_0's binary_logloss: 0.5635
[623]	valid_0's binary_logloss: 0.563482
[624]	valid_0's binary_logloss: 0.563464
[625]	valid_0's binary_logloss: 0.563454
[626]	valid_0's binary_logloss: 0.563419
[627]	valid_0's binary_logloss: 0.563363
[628]	valid_0's binary_logloss: 0.563327
[629]	valid_0's binary_logloss: 0.563242
[630]	valid_0's binary_logloss: 0.563205
[631]	valid_0's binary_logloss: 0.563172
[632]	valid_0's binary_logloss: 0.56315
[633]	valid_0's binary_logloss: 0.56311
[634]	valid_0's binary_logloss: 0.563057
[635]	valid_0's binary_logloss: 0.563028
[636]	valid_0's binary_logloss: 0.563001
[637]	valid_0's binary_logloss: 0.562972
[638]	valid_0's binary_logloss: 0.562902
[639]	valid_0's binary_logloss: 0.562885
[640]	valid_0's binary_logloss: 0.562855
[641]	valid_0's bina

...

[1422]	valid_0's binary_logloss: 0.551918
[1423]	valid_0's binary_logloss: 0.551908
[1424]	valid_0's binary_logloss: 0.551901
[1425]	valid_0's binary_logloss: 0.551894
[1426]	valid_0's binary_logloss: 0.55189
[1427]	valid_0's binary_logloss: 0.551884
[1428]	valid_0's binary_logloss: 0.551875
[1429]	valid_0's binary_logloss: 0.551871
[1430]	valid_0's binary_logloss: 0.551874
[1431]	valid_0's binary_logloss: 0.55187
[1432]	valid_0's binary_logloss: 0.551858
[1433]	valid_0's binary_logloss: 0.551842
[1434]	valid_0's binary_logloss: 0.551836
[1435]	valid_0's binary_logloss: 0.551823
[1436]	valid_0's binary_logloss: 0.551833
[1437]	valid_0's binary_logloss: 0.551838
[1438]	valid_0's binary_logloss: 0.551842
[1439]	valid_0's binary_logloss: 0.551826
[1440]	valid_0's binary_logloss: 0.551827
[1441]	valid_0's binary_logloss: 0.551828
[1442]	valid_0's binary_logloss: 0.551809
[1443]	valid_0's binary_logloss: 0.551811
[1444]	valid_0's binary_logloss: 0.551807
[1445]	valid_0's binary_logloss: 0.5

[1620]	valid_0's binary_logloss: 0.551414
[1621]	valid_0's binary_logloss: 0.551409
[1622]	valid_0's binary_logloss: 0.551408
[1623]	valid_0's binary_logloss: 0.551405
[1624]	valid_0's binary_logloss: 0.551411
[1625]	valid_0's binary_logloss: 0.551405
[1626]	valid_0's binary_logloss: 0.551389
[1627]	valid_0's binary_logloss: 0.55139
[1628]	valid_0's binary_logloss: 0.551373
[1629]	valid_0's binary_logloss: 0.551371
[1630]	valid_0's binary_logloss: 0.551369
[1631]	valid_0's binary_logloss: 0.551363
[1632]	valid_0's binary_logloss: 0.551355
[1633]	valid_0's binary_logloss: 0.551354
[1634]	valid_0's binary_logloss: 0.551347
[1635]	valid_0's binary_logloss: 0.55135
[1636]	valid_0's binary_logloss: 0.551344
[1637]	valid_0's binary_logloss: 0.551337
[1638]	valid_0's binary_logloss: 0.551334
[1639]	valid_0's binary_logloss: 0.551326
[1640]	valid_0's binary_logloss: 0.551324
[1641]	valid_0's binary_logloss: 0.551325
[1642]	valid_0's binary_logloss: 0.551322
[1643]	valid_0's binary_logloss: 0.5

[1818]	valid_0's binary_logloss: 0.551074
[1819]	valid_0's binary_logloss: 0.551085
[1820]	valid_0's binary_logloss: 0.551101
[1821]	valid_0's binary_logloss: 0.551101
[1822]	valid_0's binary_logloss: 0.551101
[1823]	valid_0's binary_logloss: 0.551107
[1824]	valid_0's binary_logloss: 0.551107
[1825]	valid_0's binary_logloss: 0.551114
[1826]	valid_0's binary_logloss: 0.551119
[1827]	valid_0's binary_logloss: 0.551119
[1828]	valid_0's binary_logloss: 0.551121
[1829]	valid_0's binary_logloss: 0.551118
[1830]	valid_0's binary_logloss: 0.551113
[1831]	valid_0's binary_logloss: 0.551116
[1832]	valid_0's binary_logloss: 0.551119
[1833]	valid_0's binary_logloss: 0.551117
[1834]	valid_0's binary_logloss: 0.551116
[1835]	valid_0's binary_logloss: 0.551117
[1836]	valid_0's binary_logloss: 0.551114
[1837]	valid_0's binary_logloss: 0.551122
[1838]	valid_0's binary_logloss: 0.551123
[1839]	valid_0's binary_logloss: 0.551112
[1840]	valid_0's binary_logloss: 0.551115
[1841]	valid_0's binary_logloss: 0

[2014]	valid_0's binary_logloss: 0.550844
[2015]	valid_0's binary_logloss: 0.550849
[2016]	valid_0's binary_logloss: 0.550846
[2017]	valid_0's binary_logloss: 0.550843
[2018]	valid_0's binary_logloss: 0.55084
[2019]	valid_0's binary_logloss: 0.55084
[2020]	valid_0's binary_logloss: 0.55085
[2021]	valid_0's binary_logloss: 0.550859
[2022]	valid_0's binary_logloss: 0.550853
[2023]	valid_0's binary_logloss: 0.550854
[2024]	valid_0's binary_logloss: 0.550847
[2025]	valid_0's binary_logloss: 0.550845
[2026]	valid_0's binary_logloss: 0.550851
[2027]	valid_0's binary_logloss: 0.55085
[2028]	valid_0's binary_logloss: 0.550842
[2029]	valid_0's binary_logloss: 0.550834
[2030]	valid_0's binary_logloss: 0.550833
[2031]	valid_0's binary_logloss: 0.550833
[2032]	valid_0's binary_logloss: 0.55083
[2033]	valid_0's binary_logloss: 0.55082
[2034]	valid_0's binary_logloss: 0.550822
[2035]	valid_0's binary_logloss: 0.550819
[2036]	valid_0's binary_logloss: 0.550804
[2037]	valid_0's binary_logloss: 0.55081

[2213]	valid_0's binary_logloss: 0.55066
[2214]	valid_0's binary_logloss: 0.550653
[2215]	valid_0's binary_logloss: 0.550645
[2216]	valid_0's binary_logloss: 0.550652
[2217]	valid_0's binary_logloss: 0.550655
[2218]	valid_0's binary_logloss: 0.550654
[2219]	valid_0's binary_logloss: 0.550655
[2220]	valid_0's binary_logloss: 0.550653
[2221]	valid_0's binary_logloss: 0.55065
[2222]	valid_0's binary_logloss: 0.550645
[2223]	valid_0's binary_logloss: 0.550649
[2224]	valid_0's binary_logloss: 0.550648
[2225]	valid_0's binary_logloss: 0.550658
[2226]	valid_0's binary_logloss: 0.550648
[2227]	valid_0's binary_logloss: 0.550648
[2228]	valid_0's binary_logloss: 0.550649
[2229]	valid_0's binary_logloss: 0.550644
[2230]	valid_0's binary_logloss: 0.550644
[2231]	valid_0's binary_logloss: 0.550641
[2232]	valid_0's binary_logloss: 0.550644
[2233]	valid_0's binary_logloss: 0.550639
[2234]	valid_0's binary_logloss: 0.550638
[2235]	valid_0's binary_logloss: 0.550638
[2236]	valid_0's binary_logloss: 0.5

## Model Results on Test

In [89]:
predictions=np.where(model.predict(X_test,num_iterations=model.best_iteration) > 0.5, 1, 0)
print(roc_auc_score(Y_test,model.predict(X_test,num_iterations=model.best_iteration)))
print(classification_report(Y_test,predictions))

0.7940092942558667
             precision    recall  f1-score   support

          0       0.72      0.72      0.72      6229
          1       0.72      0.72      0.72      6184

avg / total       0.72      0.72      0.72     12413



## Model Results on Train

In [91]:
predictions=np.where(model.predict(X_train,num_iterations=model.best_iteration) > 0.5, 1, 0)
print(roc_auc_score(Y_train,model.predict(X_train,num_iterations=model.best_iteration)))
print(classification_report(Y_train,predictions))

0.8961834591210549
             precision    recall  f1-score   support

          0       0.82      0.81      0.82     18596
          1       0.82      0.82      0.82     18641

avg / total       0.82      0.82      0.82     37237



In [ ]:
predictions=np.where(model.predict(X_test,num_iterations=model.best_iteration) > 0.5, 1, 0)
print(roc_auc_score(Y_test,model.predict(X_test,num_iterations=model.best_iteration)))
print(classification_report(Y_test,predictions))

## Save results for later ROC Curve plotting

In [68]:
pd.DataFrame(model.predict(X_test,num_iterations=model.best_iteration)).to_csv('lgb_results.csv')

## Get feature scores

In [92]:
feature_score = pd.DataFrame(model.feature_importance(), columns=['score'])
feature_score['feature'] = X.columns
feature_score = feature_score.sort_values(by=['score'], ascending=False).reset_index(drop=True)

In [94]:
feature_score.to_csv('featurescore.csv')

## Save model parameters

In [38]:
output = open('lgb_params_956.pkl', 'wb')
pickle.dump(lgb_params, output)
output.close()